# RNA Velocity Analysis of scRNAseq data from HIV infected CD4+ Tcells

Sequence data is from: PMID 30282021. In summary the authors compared HIV-infected versus uninfected CD4+ T-cells from 2 donors in a 8-12 week in vitro model using 10X Chromium V2. The goal was to simulate HIV latency and describe the transcriptomic landscape of the HIV latent reservoir.

Samples used are  the following:
1. SRR6825024 - Donor 1, Infected
2. SRR6825025 - Donor 2, Infected
3. SRR6825026 - Donor 1, Uninfected
4. SRR6825027 - Donor 2, Uninfected

This notebook processes the data after mapping to the combined human/HIV transcriptome. I took the following steps to generate raw count data (external to this notebook):

1. The genome sequence for NL43-D6-dreGFP (synthetic strain of HIV containing GFP) was appended to the human transcriptome (GRCh38).
2. "unspliced" transcripts were appended to the transcriptome.
3. The salmon index was created using the entire human genome as a decoy sequence. Used an r5.xlarge on AWS.
4. Mapping was performed with alevin for all 4 samples. Used a c5.12xlarge.
5. Downloaded data to local for further analysis.


### Load packages for using R

In [4]:
%load_ext rpy2.ipython

In [5]:
%%R
#Load Necessary Packages
library(tximeta)
library(SingleCellExperiment)

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: GenomicRanges

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: 'BiocGenerics'


R[write to console]: The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, 

## Run this next section once for each sample and change the "key" each time
Data is dumped to a python pickle for fast reloading

In [12]:
%%R
#### Note: Slow (~5min / sample) - only run once per sample

#Load data from alevin format using tximeta
key = "SRR6825027" #Set sample name here: c('SRR6825024','SRR6825025','SRR6825026','SRR6825027')
txi <- tximeta::tximeta(coldata = data.frame(
  names = key,
  files = paste0("input/", key, "/alevin/quants_mat.gz"), 
  stringsAsFactors = FALSE
), type = "alevin")

## Split the unspliced and spliced counts into two "assays" in the object
cg <- read.delim("input/for_tximeta/combined_transcript.spliced_to_unspliced.tsv",
                 header = TRUE, as.is = TRUE)

# This next line splits unspliced and spliced transcript counts - 
# if either spliced or unspliced are missing then add a zero.
txis <- splitSE(txi, cg, assayName = "counts")

#Store in sce object for conversion to anndata (Python)
txis_sce <- as(txis, "SingleCellExperiment")
assays(txis_sce) <- list(
    counts = assay(txis_sce, "spliced"),
    spliced = assay(txis_sce, "spliced"),
    unspliced = assay(txis_sce, "unspliced")
)

print(sum(assay(txis_sce, "spliced")))
print(sum(assay(txis_sce, "unspliced")))

R[write to console]: importing quantifications

R[write to console]: reading in alevin gene-level counts across cells with fishpond

R[write to console]: couldn't find matching transcriptome, returning non-ranged SummarizedExperiment



[1] 41989094
[1] 26061958


In [13]:
# Import R objects into Python
import anndata
import anndata2ri
from rpy2.robjects import r
import pickle

anndata2ri.activate()
adata = r('txis_sce')

# Dump to pickle - for fast loading later
key = r('key')[0]
with open('processed_data/'+key+'_adata_raw.pickle', 'wb') as fp:
    pickle.dump(adata,fp)

## Create trimmed version of raw files for analysis

In [2]:
#Import packages
import scanpy as sc
import sys
import numpy as np
import pickle
import scvelo as scv
import anndata
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt

#Setup for plotting
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

In [3]:
#Load samples into memory
adict = {}
for samp, key in zip(['SRR6825024','SRR6825025','SRR6825026','SRR6825027'], ['d1i','d2i','d1u','d2u']):
    with open('processed_data/'+samp+'_adata_raw.pickle', 'rb') as fp:
        adict[key] = pickle.load(fp)
    
    # Remove calculated values when present
    adict[key].uns = {}
    try:
        adict[key].obsm.pop('X_pca')
        adict[key].obsm.pop('X_tsne')
        adict[key].layers.pop('logcounts')
        adict[key].obs.drop(['sizeFactor'],axis=1,inplace=True)
    except KeyError:
        pass

In [4]:
# Plot the distribution of the read counts before trimming
def plot_readcount_pretrim(adict, key):
    logcounts = np.log10((adict[key].layers['spliced'] + adict[key].layers['unspliced']).sum(axis=1))
    y = sorted(logcounts.tolist(),reverse=True)
    x = range(len(y))
    plt.figure()
    plt.plot(x,y)
    plt.xlabel('Barcode (Cell) Rank')
    plt.ylabel('log10[readcount] (spliced + unspliced)')
    plt.savefig('figures/QC/'+key+'_readcount_pretrim.png')
    
plot_readcount_pretrim(adict, 'd1i')
plot_readcount_pretrim(adict, 'd1u')
plot_readcount_pretrim(adict, 'd2i')
plot_readcount_pretrim(adict, 'd2u')

In [5]:
# Trim (likely) doublets and empty cells. Cutoffs chosen manually
def trim_cells(adict,key,min_lcount,max_lcount):
    
    #Trim cells
    logcounts = np.log10((adict[key].layers['spliced'] + adict[key].layers['unspliced']).sum(axis=1))
    adict[key] = adict[key][(logcounts > min_lcount) & (logcounts < max_lcount), :]

    logcounts = np.log10((adict[key].layers['spliced'] + adict[key].layers['unspliced']).sum(axis=1))
    y = sorted(logcounts.tolist(),reverse=True)
    x = range(len(y))
    plt.figure()
    plt.plot(x,y)
    plt.xlabel('Barcode (Cell) Rank')
    plt.ylabel('log10[readcount] (spliced + unspliced)')
    plt.savefig('figures/QC/'+key+'_readcount_posttrim.png')
    
trim_cells(adict,'d1i', 0, 10) #No trimming
trim_cells(adict,'d1u', 0, 10) #No trimming
trim_cells(adict,'d2i', 2.8, 3.8)
trim_cells(adict,'d2u', 3.0, 4.2)

In [6]:
# Plot read counts by gene
def plot_readcount_gene(adict,key):
    temp = (adict[key].layers['spliced'])
    counts = temp.sum(axis=0).tolist()[0]
    logcounts = np.log10(temp.sum(axis=0) + 0.1).tolist()[0]
    y = sorted(logcounts,reverse=True)
    x = range(len(y))
    plt.figure()
    plt.plot(x,y)
    plt.xlabel('Gene Index')
    plt.ylabel('log10[readcount] (spliced)')
    plt.savefig('figures/QC/'+key+'_readcount_pergene_spliced')

plot_readcount_gene(adict,'d1u')
plot_readcount_gene(adict,'d1i')
plot_readcount_gene(adict,'d2u')
plot_readcount_gene(adict,'d2i')

In [7]:
#Trim genes by choose a subset that is good quality in all samples
temp_d1u = scv.pp.filter_genes(adict['d1u'], min_cells = 20, copy=True)
temp_d1u = scv.pp.filter_genes_dispersion(temp_d1u, n_top_genes=20000, copy=True)

temp_d1i = scv.pp.filter_genes(adict['d1i'], min_cells = 20, copy=True)
temp_d1i = scv.pp.filter_genes_dispersion(temp_d1i, n_top_genes=20000, copy=True)

temp_d2u = scv.pp.filter_genes(adict['d2u'], min_cells = 20, copy=True)
temp_d2u = scv.pp.filter_genes_dispersion(temp_d2u, n_top_genes=20000, copy=True)

temp_d2i = scv.pp.filter_genes(adict['d2i'], min_cells = 20, copy=True)
temp_d2i = scv.pp.filter_genes_dispersion(temp_d2i, n_top_genes=20000, copy=True)

#Take intersection of genes that pass filter and add genes of interest
genes_of_interest = ['NL43.g','ENSG00000167286.9', 'ENSG00000198851.10', 'ENSG00000160654.11']
gene_pf = (temp_d1i.var.index & temp_d1u.var.index & temp_d2u.var.index & temp_d2i.var.index) #TODO: You should only use these for the velocity fit
gene_keep = gene_pf.union(pd.Index(genes_of_interest))

#Note: The approach of restricting to the intersection will tend to yield a small number of cells with identical values. These are NOT duplicates.

Filtered out 45693 genes that are detected in less than 20 cells (spliced).
Skip filtering by dispersion since number of variables are less than `n_top_genes`
Filtered out 45124 genes that are detected in less than 20 cells (spliced).
Skip filtering by dispersion since number of variables are less than `n_top_genes`
Filtered out 45706 genes that are detected in less than 20 cells (spliced).
Skip filtering by dispersion since number of variables are less than `n_top_genes`
Filtered out 48787 genes that are detected in less than 20 cells (spliced).
Skip filtering by dispersion since number of variables are less than `n_top_genes`


In [8]:
# Plot the number of genes detected after subsetting
def plot_genes_detected(adata,key,gene_keep,min_count):
    n_genes_detected = np.squeeze((adict[key][:,gene_keep].layers['spliced'] >= min_count).A.sum(axis=1))
    
    y = sorted(n_genes_detected,reverse=True)
    x = range(len(y))
    
    plt.figure()
    plt.plot(x,y)
    plt.xlabel('Barcode (Cell) Rank')
    plt.ylabel('# Genes Detected')
    plt.savefig('figures/QC/'+key+'_genes_detected_posttrim.png')
    
plot_genes_detected(adict,'d1i',gene_keep,1)
plot_genes_detected(adict,'d1u',gene_keep,1)
plot_genes_detected(adict,'d2i',gene_keep,1)
plot_genes_detected(adict,'d2u',gene_keep,1)

In [10]:
# Merge individual anndata objects and write out
trim_ls = []
for key in ['d1u','d1i','d2u','d2i']:
    trim_ls.append(adict[key][:,gene_keep])
adata_merged = trim_ls[0].concatenate(trim_ls[1:4],uns_merge=None, batch_key='group', batch_categories=['d1u','d1i','d2u','d2i'])
adata_merged.write('processed_data/merged_adata_trimmed.h5ad')

In [21]:
# Export trimmed data including all genes
for key in ['d1u','d1i','d2u','d2i']:
    adict[key][:,gene_keep].write('processed_data/'+key+'_adata_trimmed.h5ad')

# Pooled Sample Analysis

In [11]:
#Import data
adata = anndata.read_h5ad('processed_data/merged_adata_trimmed.h5ad')
print(adata)

AnnData object with n_obs × n_vars = 24645 × 11160
    obs: 'group'
    layers: 'spliced', 'unspliced'


In [12]:
# Print fraction of spliced/unspliced
scv.utils.show_proportions(adata)
print('\n')
# Compute various useful values
scv.pp.normalize_per_cell(adata, counts_per_cell_after = 10**6, enforce=True)
sc.pp.log1p(adata, base=10)
scv.pp.neighbors(adata)
scv.tl.umap(adata)
print('\n')
print(adata)

Abundance of ['spliced', 'unspliced']: [0.68 0.32]


Normalized count data: X, spliced, unspliced.
computing neighbors
    finished (0:00:10) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)


AnnData object with n_obs × n_vars = 24645 × 11160
    obs: 'group', 'n_counts'
    var: 'gene_count_corr'
    uns: 'log1p', 'pca', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'spliced', 'unspliced'
    obsp: 'distances', 'connectivities'


In [13]:
sc.pl.umap(adata, save='_merged_B2M.png', color='ENSG00000166710.20', title='B2M')
sc.pl.umap(adata, save='_merged_CD3d.png', color='ENSG00000167286.9', title='CD3d')
sc.pl.umap(adata, save='_merged_HIV.png', color='NL43.g', title='HIV')
sc.pl.umap(adata, save='_merged_group.png', color='group', title='Donor and Infection Effects')

# Individual sample analysis

### For this section: choose a condition to run the velocity analysis by setting the variable "key"

In [3]:
# Import data
for key in ['d1i', 'd1u', 'd2u', 'd2i']: #Choose conditions for analysis here: ['d1u','d1i','d2u','d2i']
    #Read in data
    adata = anndata.read_h5ad('processed_data/'+key+'_adata_trimmed.h5ad')

    # Print fraction of spliced/unspliced
    scv.utils.show_proportions(adata)
    # Compute various useful values
    scv.pp.normalize_per_cell(adata, counts_per_cell_after = 10**6, enforce=True)
    sc.pp.log1p(adata,base=10)
    scv.pp.moments(adata, n_pcs = 30, n_neighbors = 30)
    scv.tl.umap(adata)

    #Compute velocities for each gene
    scv.tl.velocity(adata, mode = 'stochastic')

    #Ensure HIV expression is NOT used for velocity calculation
    gene_list_bool = adata.var.velocity_genes.copy()
    gene_list_bool['NL43.g'] = False
    gene_subset = adata.var_names[adata.var.velocity_genes]

    #Compute Cosine correlations
    scv.tl.velocity_graph(adata,gene_subset = gene_subset)

    adata.write('processed_data/'+key+'_adata_plots.h5ad')

Abundance of ['spliced', 'unspliced']: [0.73 0.27]
Normalized count data: X, spliced, unspliced.
computing neighbors
    finished (0:00:02) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:05) --> added 
    'Ms' and 'Mu', moments of spliced/unspliced abundances (adata.layers)
computing velocities
    finished (0:00:03) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph
    finished (0:00:31) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
Abundance of ['spliced', 'unspliced']: [0.68 0.32]
Normalized count data: X, spliced, unspliced.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:04) --> added 
    'Ms' and 'Mu', moments of spliced/unspliced abunda

## Plotting

In [14]:
for key in ['d1i', 'd1u', 'd2u', 'd2i']:
    adata = anndata.read_h5ad('processed_data/'+key+'_adata_plots.h5ad')

    scv.pl.velocity_embedding_stream(adata, basis='pca', save=key+'_HIV_pca.png', color='NL43.g', frameon=True, title='HIV')
    scv.pl.velocity_embedding_stream(adata, basis='pca', save=key+'_CD3d_pca.png', color='ENSG00000167286.9', frameon=True, title='CD3d')
    scv.pl.velocity_embedding_stream(adata, basis='pca', save=key+'_B2M_pca.png', color='ENSG00000166710.20', frameon=True, title='B2M') #Beta_2 Microglobulin

    scv.pl.velocity_embedding_stream(adata, basis = 'umap', save=key+'_HIV_umap.png', color='NL43.g', frameon=True, title='HIV')
    scv.pl.velocity_embedding_stream(adata, basis = 'umap', save=key+'_CD3d_umap.png', color='ENSG00000167286.9', frameon=True, title='CD3d')
    scv.pl.velocity_embedding_stream(adata, basis = 'umap', save=key+'_B2M_umap.png', color='ENSG00000166710.20', frameon=True, title='B2M') #Beta_2 Microglobulin

computing velocity embedding
    finished (0:00:01) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d1i_HIV_pca.png
saving figure to file ./figures/scvelo_d1i_CD3d_pca.png
saving figure to file ./figures/scvelo_d1i_B2M_pca.png
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d1i_HIV_umap.png
saving figure to file ./figures/scvelo_d1i_CD3d_umap.png
saving figure to file ./figures/scvelo_d1i_B2M_umap.png
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d1u_HIV_pca.png
saving figure to file ./figures/scvelo_d1u_CD3d_pca.png
saving figure to file ./figures/scvelo_d1u_B2M_pca.png
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure

In [7]:
# Plot histograms of expression for closer inspection
for key in ['d1i', 'd1u', 'd2u', 'd2i']:
    gene = 'ENSG00000166710.20' #Beta 2 microglobulin has an odd distribution in most samples - Can cells lose expression of B2M/MHC-1?
    hgnc = 'B2M'

    plt.figure()
    plt.hist(adata[:, gene].X.A.squeeze(),bins=50)
    plt.savefig('figures/hist_'+key+'_'+hgnc+'.png')

#### Analysis of Clusters

In [6]:
for key in ['d1i', 'd1u', 'd2u', 'd2i']:
    adata = anndata.read_h5ad('processed_data/'+key+'_adata_plots.h5ad')
    scv.tl.louvain(adata)
    scv.pl.velocity_embedding_stream(adata, basis = 'umap', save=key+'_clusters_umap.png', frameon=True, show=True)

computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d1i_clusters_umap.png
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d1u_clusters_umap.png
computing velocity embedding
    finished (0:00:03) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d2u_clusters_umap.png
computing velocity embedding
    finished (0:00:02) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file ./figures/scvelo_d2i_clusters_umap.png
